In [10]:
import numpy as np
import pyphot
from astropy.io import fits
from pyphot import unit, Filter

In [58]:
# load the filter library and define a function for calculating the spectra photometry results

from pyphot import Filter


filt_name = ['REST_FRAME/maiz-apellaniz_Johnson_U', 
             'REST_FRAME/maiz-apellaniz_Johnson_B', 
             'REST_FRAME/maiz-apellaniz_Johnson_V', 
             '2MASS/J', 
             '2MASS/H', 
             '2MASS/K']

filt_dtype = {}
filt_dtype[filt_name[0]] = 'energy'
filt_dtype[filt_name[1]] = 'energy'
filt_dtype[filt_name[2]] = 'energy'
filt_dtype[filt_name[3]] = 'photon'
filt_dtype[filt_name[4]] = 'photon'
filt_dtype[filt_name[5]] = 'photon'

filt_file = ['U.dat', 'B.dat', 'V.dat', 'J.dat', 'H.dat', 'K.dat']

filt_dir = './filters/'

def get_AB_magnitude_eazy(filt, spec_wave, spec_flux):
    f = filt
    flux = f.get_flux(spec_wave, spec_flux)
    mag = -2.5 * np.log10(flux) - f.AB_zero_mag
    return mag

In [59]:
sampler_hdu = fits.open('./pixedfit_mcmc_egs_24154.fits')

spec_z = sampler_hdu[0].header['GAL_Z']

spec_wave = sampler_hdu[5].data['wave'] / (1. + spec_z) * unit['AA']

spec_flux = sampler_hdu[5].data['tot_p50'] * unit['erg/s/cm**2/AA']

In [61]:
mags = {}

for i in range(len(filt_name)):
    filt_ascii = filt_dir + filt_file[i]
    filt_wave = np.loadtxt(filt_ascii, usecols =  [1])
    filt_tran = np.loadtxt(filt_ascii, usecols =  [2])
    filt = Filter(filt_wave, filt_tran, 
                  name = filt_name[i], 
                  dtype = filt_dtype[filt_name[i]], 
                  unit = 'Angstrom')
    mags[filt_name[i]] = get_AB_magnitude_eazy(filt = filt, spec_wave = spec_wave, spec_flux = spec_flux)

mags

In [62]:
mags

{'REST_FRAME/maiz-apellaniz_Johnson_U': 24.324348604906852,
 'REST_FRAME/maiz-apellaniz_Johnson_B': 23.339767402602263,
 'REST_FRAME/maiz-apellaniz_Johnson_V': 22.839044394055783,
 '2MASS/J': 21.71714675141208,
 '2MASS/H': 21.49444743440544,
 '2MASS/K': 21.629183341474786}

In [69]:
# if we use the pythot default filter library

lib = pyphot.get_library()

print("Library contains: ", len(lib), " filters")

filters = ['GROUND_JOHNSON_U',
           'GROUND_JOHNSON_B',
           'GROUND_JOHNSON_V', 
           '2MASS_J', 
           '2MASS_H', 
           '2MASS_Ks']

def get_AB_magnitude(filt_name, spec_wave, spec_flux):
    f = lib[filt_name]
    flux = f.get_flux(spec_wave, spec_flux)
    mag = -2.5 * np.log10(flux.value) - f.AB_zero_mag
    return mag

Library contains:  271  filters


In [70]:
# calculate

mags = {}

for i in range(len(filters)):
    mags[filters[i]] = get_AB_magnitude(filt_name = filters[i], spec_wave = spec_wave, spec_flux = spec_flux)
    
mags

{'GROUND_JOHNSON_U': 24.260517069545454,
 'GROUND_JOHNSON_B': 23.390564478107898,
 'GROUND_JOHNSON_V': 22.834965647677148,
 '2MASS_J': 21.713839553985103,
 '2MASS_H': 21.493146589824896,
 '2MASS_Ks': 21.63021314267794}